# Inspecting Ecore file for SysML v2 Metamodel

This notebook steps through a couple of different methods for loading up the SysML v2 Ecore metamodel. One with a library called pyEcore and another just using the raw lxml library. The notebook also exports the model as static metadata for PyMBE to use to correctly build model elements.

In [ ]:
import json

In [ ]:
from pyecore.resources import ResourceSet, URI
from pyecore.ecore import EClass, EAttribute, EString, EObject, EProxy, EEnum

In [ ]:
# From the quick start for PyEcore to load a file into the metaclass model

rset = ResourceSet()
kerml_resource = rset.get_resource(URI('metamodel/kerml.ecore'))
kerml_root = kerml_resource.contents[0]
rset.metamodel_registry[kerml_root.nsURI] = kerml_root

rset = ResourceSet()
sysml_resource = rset.get_resource(URI('metamodel/SysML.ecore'))
sysml_root = sysml_resource.contents[0]
rset.metamodel_registry[sysml_root.nsURI] = sysml_root

types_resource = rset.get_resource(URI('metamodel/types.ecore'))
types_root = types_resource.contents[0]
rset.metamodel_registry[types_root.nsURI] = types_root

In [ ]:
kerml_root

Determine what the fields for an EPackage in this framework are:

In [ ]:
dir(kerml_root)

In [ ]:
all_classes = sysml_root.eClassifiers
all_classes

Determine what the fields for an EClass in this framework are:

In [ ]:
dir(all_classes[0])

In [ ]:
def get_eclass(eclass_name):
    return [the_class for the_class in all_classes if the_class.name == eclass_name][0]

In [ ]:
owning_membership = [the_class for the_class in all_classes if the_class.name == "OwningMembership"][0]

In [ ]:
owning_membership

In [ ]:
owning_membership.eAttributes

In [ ]:
owning_membership.eReferences

In [ ]:
owning_membership.eAttributes[0]

In [ ]:
dir(owning_membership.eAttributes[0])

In [ ]:
owning_membership.eAttributes[0].derived

In [ ]:
attr_type = None
if isinstance(owning_membership.eAttributes[0].eType, EProxy):
    attr_type = owning_membership.eAttributes[0].eType.force_resolve()
else:
    attr_type = owning_membership.eAttributes[0].eType

In [ ]:
attr_type.name

In [ ]:
[super_type.name for super_type in all_classes[1].eAllSuperTypes()]

In [ ]:
owning_membership.eAllAttributes()

In [ ]:
owning_membership.eAllReferences()

In [ ]:
def element_has_feature_by_name(element, name):
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    for ref in all_refs:
        if ref.name == name:
            return True
        
    for att in all_atts:
        if att.name == name:
            return True
        
    return False

In [ ]:
element_has_feature_by_name(owning_membership, "relatedElement")

In [ ]:
def layout_features(element):
    
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    local_refs = element.eReferences
    local_atts = element.eAttributes
    
    feature_dict = {
        ref.name: {
            'inherited': ref not in local_refs,
            'derived': ref.derived,
            'kind': ref.eType.force_resolve().name if isinstance(ref.eType, EProxy) else ref.eType.name,
            'is_reference': True,
            'lower_bound': ref.lowerBound,
            'upper_bound': ref.upperBound,
        }
        for ref in all_refs
    } | {
        att.name: {
            'inherited': att not in local_atts,
            'derived': att.derived,
            'kind': att.eType.force_resolve().name if isinstance(att.eType, EProxy) else att.eType.name,
            'is_reference': False,
            'lower_bound': att.lowerBound,
            'upper_bound': att.upperBound,
        }
        for att in all_atts
    }
    
    return feature_dict

In [ ]:
layout_features(owning_membership)

In [ ]:
def get_primary_attributes(eclass_name):
    eclass = get_eclass(eclass_name)
    
    all_features = layout_features(eclass)
    
    primary_features = {name: feature for name, feature in all_features.items() if not feature['derived']}
    
    return primary_features

In [ ]:
def get_derived_attributes(eclass_name):
    eclass = get_eclass(eclass_name)
    
    all_features = layout_features(eclass)
    
    primary_features = {name: feature for name, feature in all_features.items() if feature['derived']}
    
    return primary_features

In [ ]:
layout_features(get_eclass("OwningMembership"))

In [ ]:
layout_features(get_eclass("PartUsage"))

In [ ]:
def find_all_sub_types(element):
    # go through all of the meta-elements in the Ecore and find the ones that have the 
    # element given as an argument as one of their super types
    
    sub_types = []
    
    all_classes = sysml_root.eClassifiers
    for clz in all_classes:
        if not isinstance(clz, EEnum):
            if element in clz.eAllSuperTypes():
                sub_types.append(clz)
            
    return sub_types

In [ ]:
find_all_sub_types(get_eclass("OwningMembership"))

In [ ]:
get_primary_attributes("PartDefinition")

In [ ]:
get_derived_attributes("PartUsage")

In [ ]:
get_primary_attributes("OwningMembership")

In [ ]:
get_primary_attributes("Membership")

In [ ]:
get_primary_attributes("Specialization")

In [ ]:
get_primary_attributes("FeatureTyping")

In [ ]:
attributes_dict = {k.name:get_primary_attributes(k.name) for k in all_classes if isinstance(k, EClass)}

In [ ]:
# want layout_features on all metaclasses in kerml, sysml
sysml_classes = [k for k in sysml_root.eClassifiers if isinstance(k, EClass)]
kerml_classes = [k for k in kerml_root.eClassifiers if isinstance(k, EClass)] 

sysml_classes[2].name

In [ ]:
with open("metamodel/attribute_metadata.json","w") as file1:
    all_classes = [k for k in sysml_root.eClassifiers if isinstance(k, EClass)] + \
        [k for k in kerml_root.eClassifiers if isinstance(k, EClass)]
    
    metamodel_data = {clz.name: layout_features(clz) for clz in all_classes}
    
    json.dump(metamodel_data, file1)